### Load tensorflow

In [1]:
import tensorflow as tf

In [2]:
#Reset Default graph - Needed only for Jupyter notebook
tf.reset_default_graph()

### Collect Data

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('./prices.csv')

### Check all columns in the dataset

In [5]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [6]:
data=data.drop(['date','symbol'], axis=1)

In [7]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


In [8]:
data.describe()

,open,close,low,high,volume
count,851264.000000,851264.000000,851264.000000,851264.000000,8.512640e+05
mean,70.836986,70.857109,70.118414,71.543476,5.415113e+06
std,83.695876,83.689686,82.877294,84.465504,1.249468e+07
min,0.850000,0.860000,0.830000,0.880000,0.000000e+00
25%,33.840000,33.849998,33.480000,34.189999,1.221500e+06
50%,52.770000,52.799999,52.230000,53.310001,2.476250e+06
75%,79.879997,79.889999,79.110001,80.610001,5.222500e+06
max,1584.439941,1578.130005,1549.939941,1600.930054,8.596434e+08


### Consider only first 1000 rows in the dataset for building feature set and target set

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 5 columns):
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5)
memory usage: 32.5 MB


In [10]:
data=data.iloc[:1000]

In [11]:
data.shape

(1000, 5)

### Divide the data into train and test sets

In [12]:
F=data[['open','low','high','volume']]
T=data['close']

In [13]:
from sklearn.model_selection import train_test_split
F_train, F_test, T_train, T_test = train_test_split(F, T, test_size=0.30, random_state=1)

## Building the graph in tensorflow

1.Define input data placeholders

In [14]:
#Input features
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')

#Normalize the data
x_n = tf.nn.l2_normalize(x,1)

#Actual Prices
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

2.Define Weights and Bias

In [15]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

Instructions for updating:
Colocations handled automatically by placer.


3.Prediction

In [16]:
y = tf.add(tf.matmul(x_n,W),b,name='output')

4.Loss (Cost) Function [Mean square error]

In [17]:
loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [18]:
train_op = tf.train.GradientDescentOptimizer(0.03).minimize(loss)

## Execute the Graph for 100 epochs and observe the loss

In [19]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

In [20]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op,loss],feed_dict={x:F_train, y_:T_train})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

Training loss at step:  0  is  8001.065
Training loss at step:  10  is  3911.1157
Training loss at step:  20  is  3593.8867
Training loss at step:  30  is  3569.2832
Training loss at step:  40  is  3567.3726
Training loss at step:  50  is  3567.2246
Training loss at step:  60  is  3567.2131
Training loss at step:  70  is  3567.2136
Training loss at step:  80  is  3567.2117
Training loss at step:  90  is  3567.2122


### Get the shapes and values of W and b

Hint: Use sess.run(W) to get W.

In [21]:
sess.run(W)

array([[2.6559869e-03],
       [2.6320121e-03],
       [2.6763668e-03],
       [3.3293488e+01]], dtype=float32)

In [22]:
sess.run(b)

array([33.2935], dtype=float32)

### Find the Absolute mean square loss difference between training and testing loss.

In [25]:
testing_epochs=100

In [27]:
test_op = tf.train.GradientDescentOptimizer(0.03).minimize(loss)

In [28]:

for epoch in range(testing_epochs):
            
    #Calculate test_op and loss
    _, test_loss = sess.run([test_op,loss],feed_dict={x:F_test, y_:T_test})
    
    if epoch % 10 == 0:
        print ('Testing loss at step: ', epoch, ' is ', test_loss)

Testing loss at step:  0  is  3018.3823
Testing loss at step:  10  is  3018.018
Testing loss at step:  20  is  3017.9878
Testing loss at step:  30  is  3017.9846
Testing loss at step:  40  is  3017.9827
Testing loss at step:  50  is  3017.986
Testing loss at step:  60  is  3017.9834
Testing loss at step:  70  is  3017.986
Testing loss at step:  80  is  3017.9868
Testing loss at step:  90  is  3017.9868


In [29]:
train_loss

3567.213

In [30]:
test_loss

3017.9868

In [31]:
print(abs(train_loss-test_loss))

549.2261


In [32]:
sess.close()

### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [33]:
#Initialize Sequential Graph (model)
model = tf.keras.models.Sequential()

#Normalize input data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1))

#set up learning rate 
sgd = tf.keras.optimizers.SGD(lr = 0.003)

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer= sgd, loss='mse')

Instructions for updating:
Use tf.cast instead.


### Execute the model

In [34]:
model.fit(F_train,T_train,validation_data=(F_test,T_test), epochs=100)

Train on 700 samples, validate on 300 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
700/700 [==============================] - 0s 295us/sample - loss: 3135.6966 - val_loss: 8956.6620
Epoch 2/100
700/700 [==============================] - 0s 39us/sample - loss: 1066.6469 - val_loss: 4801.8039
Epoch 3/100
700/700 [==============================] - 0s 43us/sample - loss: 1158.3413 - val_loss: 13386.6196
Epoch 4/100
700/700 [==============================] - 0s 41us/sample - loss: 1702.3314 - val_loss: 201.5317
Epoch 5/100
700/700 [==============================] - 0s 46us/sample - loss: 1366.7908 - val_loss: 6866.6709
Epoch 6/100
700/700 [==============================] - 0s 47us/sample - loss: 1232.0117 - val_loss: 134.1644
Epoch 7/100
700/700 [==============================] - 0s 39us/sample - loss: 1677.5138 - val_loss: 1036.0292
Epoch 8/100
700/700 [==============================] - 0s 40us/sample - loss: 1217.8144 - val_loss: 84.7456
Epoch 9/100
700/700 [=======

Epoch 73/100
700/700 [==============================] - 0s 38us/sample - loss: 1431.0519 - val_loss: 656.4617
Epoch 74/100
700/700 [==============================] - 0s 41us/sample - loss: 1030.2531 - val_loss: 30.4050
Epoch 75/100
700/700 [==============================] - 0s 39us/sample - loss: 891.5887 - val_loss: 270.3419
Epoch 76/100
700/700 [==============================] - 0s 39us/sample - loss: 1037.8149 - val_loss: 265.6878
Epoch 77/100
700/700 [==============================] - 0s 44us/sample - loss: 862.1677 - val_loss: 39.0890
Epoch 78/100
700/700 [==============================] - 0s 40us/sample - loss: 1068.6136 - val_loss: 247.9492
Epoch 79/100
700/700 [==============================] - 0s 34us/sample - loss: 1043.2289 - val_loss: 305.8956
Epoch 80/100
700/700 [==============================] - 0s 44us/sample - loss: 1001.3946 - val_loss: 832.0366
Epoch 81/100
700/700 [==============================] - 0s 43us/sample - loss: 844.0724 - val_loss: 1518.1275
Epoch 82/100
7

In [35]:
model.get_weights()

[array([10.43896   , -1.023367  ,  0.817872  ,  0.46716166], dtype=float32),
 array([-2.9270864,  0.2770029, -0.2277575,  9.904574 ], dtype=float32),
 array([6.6457123e+01, 6.5782745e+01, 6.7101028e+01, 5.5219340e+06],
       dtype=float32),
 array([3.4787126e+03, 3.4156914e+03, 3.5124534e+03, 1.8810145e+14],
       dtype=float32),
 array([[ 8.522766  ],
        [-0.83911884],
        [ 0.66844034],
        [ 9.007692  ]], dtype=float32),
 array([15.589224], dtype=float32)]

### Classification using Keras 

In [36]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_v1 (Batc (None, 4)                 16        
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 21
Trainable params: 13
Non-trainable params: 8
_________________________________________________________________


In [37]:
sess.close()

### Load the given Iris data using pandas (Iris.csv)

In [38]:
iris_df = pd.read_csv('Iris.csv')

In [39]:
iris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
Id               150 non-null int64
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), int64(1), object(1)
memory usage: 7.1+ KB


### Splitting the data into feature set and target set

In [40]:
iris_df = iris_df.drop('Id', axis=1)

In [41]:
Y = iris_df['Species']
X = iris_df.drop('Species', axis = 1)

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [42]:
Y_dummies = pd.DataFrame(pd.get_dummies(Y, sparse=True).values)

In [43]:
Y_dummies.shape

(150, 3)

### Divide the dataset into Training and test (70:30)

In [44]:
trainX, testX,trainY,testY = train_test_split(X, Y_dummies, test_size=0.3, random_state=100)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [45]:
#Initialize the sequential model
model_iris = tf.keras.models.Sequential()

#Normalize the data
model_iris.add(tf.keras.layers.BatchNormalization())

#Add Dense layer which provides 10 outputs after applying softmax
model_iris.add(tf.keras.layers.Dense(10, activation='sigmoid', input_dim=4))

#Add Dense layer which provides 10 outputs after applying softmax
model_iris.add(tf.keras.layers.Dense(8, activation='sigmoid'))

#sgd = tf.keras.optimizers.SGD(lr = 0.003)

#Add Dense layer which provides 10 outputs after applying softmax
model_iris.add(tf.keras.layers.Dense(3, activation='softmax'))

In [46]:
#Complile the model
model_iris.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

### Fitting the model and predicting 

In [47]:
model_iris.fit(trainX.values,trainY.values,validation_data=(testX,testY), epochs=1000)

Train on 105 samples, validate on 45 samples
Epoch 1/1000
105/105 [==============================] - 0s 2ms/sample - loss: 1.1388 - acc: 0.3714 - val_loss: 1.1587 - val_acc: 0.2444
Epoch 2/1000
105/105 [==============================] - 0s 85us/sample - loss: 1.1347 - acc: 0.3714 - val_loss: 1.1534 - val_acc: 0.2444
Epoch 3/1000
105/105 [==============================] - 0s 80us/sample - loss: 1.1303 - acc: 0.3714 - val_loss: 1.1509 - val_acc: 0.2444
Epoch 4/1000
105/105 [==============================] - 0s 118us/sample - loss: 1.1274 - acc: 0.3714 - val_loss: 1.1479 - val_acc: 0.2444
Epoch 5/1000
105/105 [==============================] - 0s 104us/sample - loss: 1.1247 - acc: 0.3714 - val_loss: 1.1437 - val_acc: 0.2444
Epoch 6/1000
105/105 [==============================] - 0s 109us/sample - loss: 1.1214 - acc: 0.3714 - val_loss: 1.1410 - val_acc: 0.2444
Epoch 7/1000
105/105 [==============================] - 0s 76us/sample - loss: 1.1187 - acc: 0.3714 - val_loss: 1.1382 - val_acc: 0

105/105 [==============================] - 0s 85us/sample - loss: 1.0709 - acc: 0.3714 - val_loss: 1.0863 - val_acc: 0.2667
Epoch 60/1000
105/105 [==============================] - 0s 94us/sample - loss: 1.0696 - acc: 0.3714 - val_loss: 1.0857 - val_acc: 0.2667
Epoch 61/1000
105/105 [==============================] - 0s 113us/sample - loss: 1.0692 - acc: 0.3714 - val_loss: 1.0856 - val_acc: 0.2667
Epoch 62/1000
105/105 [==============================] - 0s 99us/sample - loss: 1.0695 - acc: 0.3714 - val_loss: 1.0839 - val_acc: 0.2889
Epoch 63/1000
105/105 [==============================] - 0s 94us/sample - loss: 1.0683 - acc: 0.3810 - val_loss: 1.0838 - val_acc: 0.2889
Epoch 64/1000
105/105 [==============================] - 0s 146us/sample - loss: 1.0680 - acc: 0.3810 - val_loss: 1.0843 - val_acc: 0.2667
Epoch 65/1000
105/105 [==============================] - 0s 156us/sample - loss: 1.0672 - acc: 0.3810 - val_loss: 1.0836 - val_acc: 0.2889
Epoch 66/1000
105/105 [======================

Epoch 118/1000
105/105 [==============================] - 0s 99us/sample - loss: 1.0439 - acc: 0.6952 - val_loss: 1.0550 - val_acc: 0.5778
Epoch 119/1000
105/105 [==============================] - 0s 118us/sample - loss: 1.0428 - acc: 0.6952 - val_loss: 1.0550 - val_acc: 0.5778
Epoch 120/1000
105/105 [==============================] - 0s 85us/sample - loss: 1.0439 - acc: 0.6857 - val_loss: 1.0539 - val_acc: 0.5778
Epoch 121/1000
105/105 [==============================] - 0s 128us/sample - loss: 1.0422 - acc: 0.6952 - val_loss: 1.0533 - val_acc: 0.5778
Epoch 122/1000
105/105 [==============================] - 0s 132us/sample - loss: 1.0436 - acc: 0.6952 - val_loss: 1.0527 - val_acc: 0.5778
Epoch 123/1000
105/105 [==============================] - 0s 113us/sample - loss: 1.0443 - acc: 0.6857 - val_loss: 1.0517 - val_acc: 0.5778
Epoch 124/1000
105/105 [==============================] - 0s 85us/sample - loss: 1.0416 - acc: 0.6952 - val_loss: 1.0518 - val_acc: 0.5778
Epoch 125/1000
105/105 

Epoch 177/1000
105/105 [==============================] - 0s 71us/sample - loss: 1.0143 - acc: 0.6952 - val_loss: 1.0238 - val_acc: 0.6000
Epoch 178/1000
105/105 [==============================] - 0s 66us/sample - loss: 1.0123 - acc: 0.6952 - val_loss: 1.0231 - val_acc: 0.6000
Epoch 179/1000
105/105 [==============================] - 0s 71us/sample - loss: 1.0128 - acc: 0.6952 - val_loss: 1.0225 - val_acc: 0.6000
Epoch 180/1000
105/105 [==============================] - 0s 66us/sample - loss: 1.0120 - acc: 0.6952 - val_loss: 1.0216 - val_acc: 0.6000
Epoch 181/1000
105/105 [==============================] - 0s 76us/sample - loss: 1.0125 - acc: 0.6667 - val_loss: 1.0218 - val_acc: 0.6000
Epoch 182/1000
105/105 [==============================] - 0s 94us/sample - loss: 1.0091 - acc: 0.6952 - val_loss: 1.0204 - val_acc: 0.6000
Epoch 183/1000
105/105 [==============================] - 0s 80us/sample - loss: 1.0142 - acc: 0.6952 - val_loss: 1.0206 - val_acc: 0.6000
Epoch 184/1000
105/105 [===

Epoch 236/1000
105/105 [==============================] - 0s 71us/sample - loss: 0.9760 - acc: 0.6952 - val_loss: 0.9819 - val_acc: 0.6000
Epoch 237/1000
105/105 [==============================] - 0s 71us/sample - loss: 0.9750 - acc: 0.6952 - val_loss: 0.9816 - val_acc: 0.6000
Epoch 238/1000
105/105 [==============================] - 0s 71us/sample - loss: 0.9788 - acc: 0.6952 - val_loss: 0.9803 - val_acc: 0.6000
Epoch 239/1000
105/105 [==============================] - 0s 76us/sample - loss: 0.9809 - acc: 0.6571 - val_loss: 0.9799 - val_acc: 0.6000
Epoch 240/1000
105/105 [==============================] - 0s 80us/sample - loss: 0.9727 - acc: 0.6952 - val_loss: 0.9790 - val_acc: 0.6000
Epoch 241/1000
105/105 [==============================] - 0s 66us/sample - loss: 0.9733 - acc: 0.6857 - val_loss: 0.9797 - val_acc: 0.6000
Epoch 242/1000
105/105 [==============================] - 0s 76us/sample - loss: 0.9730 - acc: 0.6952 - val_loss: 0.9789 - val_acc: 0.6000
Epoch 243/1000
105/105 [===

...

Epoch 708/1000
105/105 [==============================] - 0s 99us/sample - loss: 0.5751 - acc: 0.6762 - val_loss: 0.5535 - val_acc: 0.6222
Epoch 709/1000
105/105 [==============================] - 0s 85us/sample - loss: 0.5718 - acc: 0.6952 - val_loss: 0.5518 - val_acc: 0.6222
Epoch 710/1000
105/105 [==============================] - 0s 85us/sample - loss: 0.5654 - acc: 0.7238 - val_loss: 0.5504 - val_acc: 0.6444
Epoch 711/1000
105/105 [==============================] - 0s 80us/sample - loss: 0.5675 - acc: 0.7238 - val_loss: 0.5506 - val_acc: 0.6444
Epoch 712/1000
105/105 [==============================] - 0s 94us/sample - loss: 0.5788 - acc: 0.7238 - val_loss: 0.5506 - val_acc: 0.6222
Epoch 713/1000
105/105 [==============================] - 0s 104us/sample - loss: 0.5934 - acc: 0.6857 - val_loss: 0.5498 - val_acc: 0.6444
Epoch 714/1000
105/105 [==============================] - 0s 99us/sample - loss: 0.5617 - acc: 0.7238 - val_loss: 0.5500 - val_acc: 0.6222
Epoch 715/1000
105/105 [==

105/105 [==============================] - 0s 71us/sample - loss: 0.5371 - acc: 0.7810 - val_loss: 0.5250 - val_acc: 0.6889
Epoch 767/1000
105/105 [==============================] - 0s 80us/sample - loss: 0.5422 - acc: 0.7810 - val_loss: 0.5250 - val_acc: 0.6889
Epoch 768/1000
105/105 [==============================] - 0s 71us/sample - loss: 0.5366 - acc: 0.7714 - val_loss: 0.5243 - val_acc: 0.6889
Epoch 769/1000
105/105 [==============================] - 0s 71us/sample - loss: 0.5399 - acc: 0.7810 - val_loss: 0.5252 - val_acc: 0.6444
Epoch 770/1000
105/105 [==============================] - 0s 66us/sample - loss: 0.5400 - acc: 0.7429 - val_loss: 0.5253 - val_acc: 0.6444
Epoch 771/1000
105/105 [==============================] - 0s 66us/sample - loss: 0.5437 - acc: 0.7333 - val_loss: 0.5246 - val_acc: 0.6444
Epoch 772/1000
105/105 [==============================] - 0s 71us/sample - loss: 0.5394 - acc: 0.7333 - val_loss: 0.5238 - val_acc: 0.6444
Epoch 773/1000
105/105 [==================

105/105 [==============================] - 0s 71us/sample - loss: 0.5439 - acc: 0.7810 - val_loss: 0.5056 - val_acc: 0.6889
Epoch 826/1000
105/105 [==============================] - 0s 71us/sample - loss: 0.5322 - acc: 0.7524 - val_loss: 0.5059 - val_acc: 0.6667
Epoch 827/1000
105/105 [==============================] - 0s 71us/sample - loss: 0.5220 - acc: 0.7619 - val_loss: 0.5061 - val_acc: 0.6444
Epoch 828/1000
105/105 [==============================] - 0s 66us/sample - loss: 0.5290 - acc: 0.7429 - val_loss: 0.5046 - val_acc: 0.6889
Epoch 829/1000
105/105 [==============================] - 0s 66us/sample - loss: 0.5166 - acc: 0.7714 - val_loss: 0.5046 - val_acc: 0.6889
Epoch 830/1000
105/105 [==============================] - 0s 66us/sample - loss: 0.5205 - acc: 0.7524 - val_loss: 0.5042 - val_acc: 0.6889
Epoch 831/1000
105/105 [==============================] - 0s 66us/sample - loss: 0.5177 - acc: 0.7524 - val_loss: 0.5039 - val_acc: 0.6889
Epoch 832/1000
105/105 [==================

105/105 [==============================] - 0s 80us/sample - loss: 0.5127 - acc: 0.7619 - val_loss: 0.4866 - val_acc: 0.8889
Epoch 885/1000
105/105 [==============================] - 0s 80us/sample - loss: 0.5082 - acc: 0.8476 - val_loss: 0.4852 - val_acc: 0.9111
Epoch 886/1000
105/105 [==============================] - 0s 66us/sample - loss: 0.5015 - acc: 0.8857 - val_loss: 0.4853 - val_acc: 0.9111
Epoch 887/1000
105/105 [==============================] - 0s 76us/sample - loss: 0.5014 - acc: 0.8571 - val_loss: 0.4850 - val_acc: 0.9111
Epoch 888/1000
105/105 [==============================] - 0s 71us/sample - loss: 0.5095 - acc: 0.8857 - val_loss: 0.4848 - val_acc: 0.9111
Epoch 889/1000
105/105 [==============================] - 0s 71us/sample - loss: 0.4970 - acc: 0.8857 - val_loss: 0.4845 - val_acc: 0.9111
Epoch 890/1000
105/105 [==============================] - 0s 71us/sample - loss: 0.5086 - acc: 0.8857 - val_loss: 0.4849 - val_acc: 0.9111
Epoch 891/1000
105/105 [==================

Epoch 943/1000
105/105 [==============================] - 0s 76us/sample - loss: 0.4916 - acc: 0.8667 - val_loss: 0.4748 - val_acc: 0.8889
Epoch 944/1000
105/105 [==============================] - 0s 76us/sample - loss: 0.4841 - acc: 0.8381 - val_loss: 0.4745 - val_acc: 0.8889
Epoch 945/1000
105/105 [==============================] - 0s 71us/sample - loss: 0.4829 - acc: 0.8190 - val_loss: 0.4733 - val_acc: 0.8889
Epoch 946/1000
105/105 [==============================] - 0s 66us/sample - loss: 0.4872 - acc: 0.8667 - val_loss: 0.4733 - val_acc: 0.8889
Epoch 947/1000
105/105 [==============================] - 0s 61us/sample - loss: 0.4881 - acc: 0.8571 - val_loss: 0.4727 - val_acc: 0.9111
Epoch 948/1000
105/105 [==============================] - 0s 71us/sample - loss: 0.4903 - acc: 0.8381 - val_loss: 0.4716 - val_acc: 0.9111
Epoch 949/1000
105/105 [==============================] - 0s 71us/sample - loss: 0.4830 - acc: 0.8762 - val_loss: 0.4722 - val_acc: 0.9111
Epoch 950/1000
105/105 [===

### Report Accuracy of the predicted values

In [48]:
model_iris.evaluate(testX,testY)

45/45 [==============================] - 0s 77us/sample - loss: 0.4590 - acc: 0.9333


[0.45903802381621467, 0.93333334]